In [6]:
import os
import requests
from langchain.chat_models.openai import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from pydantic import BaseModel, Field
from typing import Type


alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description=(
            "The query you will search for. Example query: Stock Market Symbol"
            " for Apple Company"
        )
    )


class StockMarketSymbolSearchTool(BaseTool):
    name: Type[str] = "StockMarketSymbolSearchTool"
    description: Type[
        str
    ] = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = (
        StockMarketSymbolSearchToolArgsSchema
    )

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanySymbolArgsSchema(BaseModel):
    symbol: str = Field(
        description="The stock symbol of the company. Example: AAPL,TSLA"
    )


class CompanyOverviewTool(BaseTool):
    name: Type[str] = "CompanyOverviewTool"
    description: Type[
        str
    ] = """
    Use this tool to get an overview of the financials of the company.
    You should enter a stock symbol.
    
    """
    args_schema: Type[CompanySymbolArgsSchema] = CompanySymbolArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name: Type[str] = "CompanyIncomeStatementTool"
    description: Type[
        str
    ] = """
    Use this tool to get the income statement of the financials of the company.
    You should enter a stock symbol.
    
    """
    args_schema: Type[CompanySymbolArgsSchema] = CompanySymbolArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]


class CompanyStockPerformanceTool(BaseTool):
    name: Type[str] = "CompanyStockPerformanceTool"
    description: Type[
        str
    ] = """
    Use this to get the daily performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanySymbolArgsSchema] = CompanySymbolArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,
)

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

prompt = (
    "Give me financial information on Cloudflare's stock, considering its"
    " financials, income statements and stock performance help me analyze if"
    " it's a potential good investment."
)

agent.invoke(prompt)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Cloudflare Company'}`


Cloudflare NET shares have gained 12.6% in the year-to-date (YTD) period. However, NET stock has underperformed the Zacks Internet Software industry, Zacks Computer and Technology sector and the S ... About NET. Cloudflare, Inc. operates as a cloud services provider that delivers a range of services to businesses worldwide. The company provides an integrated cloud-based security solution to secure a range of combination of platforms, including public cloud, private cloud, on-premise, software-as-a-service applications, and IoT devices; and website and application security products ... Cloudflare is up 16.5% since the beginning of the year, but at $92.49 per share, it is still trading 14.3% below its 52-week high of $107.92 from February 2024. Discover the best brokers for stock trading and ordering other assets! On Tuesday 10/08/2024 the closing price of the Cloudflare share was $8

{'input': "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment.",
 'output': "### Cloudflare Inc. Financial Analysis:\n\n#### Company Overview:\n- **Symbol:** NET\n- **Sector:** Technology\n- **Industry:** Services-Prepackaged Software\n- **Market Capitalization:** $31.94 billion\n- **Revenue (TTM):** $1.48 billion\n- **Profit Margin:** -6.9%\n- **Operating Margin (TTM):** -8.65%\n- **Return on Assets (TTM):** -3.59%\n- **Return on Equity (TTM):** -13.4%\n- **Official Website:** [Cloudflare](https://www.cloudflare.com)\n\n#### Income Statement:\n- **Latest Quarter:** June 30, 2024\n- **Revenue:** $1.48 billion\n- **Gross Profit:** $742.63 million\n- **Operating Income:** -$185.49 million\n- **Net Income:** -$183.95 million\n- **EPS:** -$0.3\n\n#### Stock Performance:\n- **Current Price:** $93.80\n- **52-Week High:** $116\n- **52-Week Low:** $53.88\n- **50-Day Moving 